# Streaming 
In this exercise we want to test our keyword spotter on self recorded wav files containing some keywords. 

In [ ]:
import os
import tensorflow as tf
import librosa
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt

from config import *
from utility import smoothing, augment_audio

### Pre-trained model

In [ ]:
## Load a pretrained model, you should choose a good one here out of the ones you have trained
checkpoint_path = './output/2022_05_02_13_29_15/saved_models/' 

In [ ]:
## Load the model
if os.path.isdir(checkpoint_path):
    model = tf.keras.models.load_model(checkpoint_path)
    print('model loaded successfully')
else:
    print('could not find model')

In [ ]:
# Print summary of trained model
model.summary()

### Deployment of keyword spotter

In [ ]:
## Applying the kws to a given 1s audio chunk of data and returns a dictionary with the calculated scores
def apply_spotter(data=[], model=model):
    assert len(data)==fs
    mfccs = augment_audio(mode='',sig=data, fs=fs, l=l, s=s, n_mfccs=n_mfccs).reshape(1,49,40)
    prediction = model.predict(mfccs)[0]
    return {k:prediction[i] for i,k in enumerate(kws_all)}

In [ ]:
## Load a recording containing some keywords. I provided this one but its more fun to try on your own recording
## I said: yes, no, what am I doing, go with short breaks in between
path_to_recording = './media/test.wav'

audio = librosa.load(path_to_recording, sr=16000)[0]
ipd.Audio(audio, rate=16000)


In [ ]:
## Divide the stream into chunks to simulate real streaming
chunksize = 4000

audio_chunked = np.array([ audio[i*chunksize:(i+1)*chunksize] for i in range(0,len(audio)//chunksize)] )
audio_chunked.shape

## Exercise

1. Create a $1s$ long audio array. For each chunk in audio_chunked, update the $1s$ long audio array and pass it to the apply_spotter function to evaluate the keyword spotter. Save the resulting probability scores into a vector "all_scores". Each entry of "all_scores" should contain $12$ probabilities, one for each keyword.
2. Plot the probabilities for all classes over time. How would you decide when a keyword was said based on the scores?
3. Understand what the smoothing function above does and use it to smooth the "all_scores" vector. Visualize the smoothed results. Are they better? Why?
4. Execute the notebook again, but with your own recording. You can use **audacity** to create a $.wav$ file. 

## Hints

- Think about how you need to adjust the $1s$ long audio snippet each time a new chunk arrives.

In [ ]:
ipd.Image('./media/streaming_kws.png')

## Solution

### E1

We drop the oldest chunk (left part of the input vector) and append the new chunk.

In [ ]:
## Apply the model to the audio stream
audio_1s = np.zeros(16000)
all_scores = []

for chunk in audio_chunked:
    
    ## Throw out the oldest chunk
    audio_1s[0:16000-chunksize] = audio_1s[chunksize:]
    
    ## Append the latest chunk
    audio_1s[16000-chunksize:] = chunk
    
    ## Apply the spotter
    scores = apply_spotter(audio_1s)
    
    ## Save scores for plotting
    all_scores.append([scores[key] for key in scores])

### E2 &  E3
We use a threshold whose value balances between false positives and false negatives. The problem is, there are some high but small width peaks that are obviously wrong. To avoid a false positive we also use the width of the peaks to decide wether a keyword was spoken. For example, this can  be done by using the area under the curve over a fixed amount of past time frames. The result (fig. 2) shows, that the narrow peaks can now be avoided by choosing a sufficiently high threshold.

In [ ]:
## plot the output probabilities
smoothing_level = 4

colors = ['red', 'green', 'blue', 'black', 'orange', 'brown', 'yellow', 'red', 'green', 'blue', 'black']
lines = ["-", "--", "-.", ":", "-", "--", "-.", ":", "-", "--", "-.", ":", "-", "--", "-.", ":"]

fig, axs = plt.subplots(2,)
fig.set_size_inches(18.5, 10.5)

hw_prob_smoothed = np.array(all_scores)

for i in range(len(hw_prob_smoothed[0])):
    axs[0].plot(hw_prob_smoothed[:,i], linestyle=lines[i])
    axs[0].axhline(y=.7, color='r')    
    
    axs[1].plot(smoothing(hw_prob_smoothed, smoothing_level)[:,i], label=kws_all[i], linestyle=lines[i])
    axs[1].axhline(y=.55, color='r')    

plt.legend()

In [ ]:
ipd.Audio(audio, rate=16000)